In [ ]:
cd '/kaggle/input/elliptic-data-set/elliptic_bitcoin_dataset/'

In [ ]:
import kmapper as km
import community as ct
from sklearn.model_selection import learning_curve
from sklearn.ensemble import RandomForestClassifier

mapper = km.KeplerMapper(verbose=2)

import sklearn
import matplotlib.pyplot as plt
from kmapper import jupyter
from sklearn.decomposition import PCA
from kmapper.cover import Cover
from sklearn import ensemble
from sklearn.cross_decomposition import PLSRegression

import pandas as pd
import numpy as np

from sklearn.neighbors import KernelDensity

from sklearn.metrics import roc_auc_score

from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import LocalOutlierFactor
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier

from sklearn.decomposition import PCA
from sklearn.neighbors import NeighborhoodComponentsAnalysis
from sklearn.cross_decomposition import PLSRegression
from sklearn.kernel_approximation import RBFSampler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

1. Chamando todos os dados
2. Definindo X_train e X_test
3. Arrumando valores de target e coluna que usava palavra reservada

In [ ]:
df_classes = pd.read_csv('elliptic_txs_classes.csv')
df_features = pd.read_csv('elliptic_txs_features.csv')
df_transaction_edge = pd.read_csv('elliptic_txs_edgelist.csv')


merged_df = df_classes.merge(df_features, left_on = 'txId', right_on = '230425980')
merged_df = merged_df.rename(columns = {'class' : 'target', '1' : 'time_step'})


merged_df = merged_df.rename(columns = {'class' : 'target', '1' : 'time_step'})
merged_df['target'] = merged_df['target'].replace({'1': 1, '2': 0, 'unknown' : -1})
merged_df = merged_df.drop(['txId', '230425980'], axis = 1)

In [ ]:
X_train = merged_df.query('time_step < 35 and target != -1').drop(['target'], axis =1)
y_train = merged_df.query('time_step < 35 and target != -1')['target']
unlabeled_df_train = merged_df.query('time_step < 35 and target == -1').drop(['target'], axis =1)


X_test = merged_df.query('time_step >= 35 and target != -1').drop(['target'], axis = 1)
y_test = merged_df.query('time_step >= 35 and target != -1')['target']

# Lentes usadas em projeções para estudo por análise topológica
* Confirmamos visualmente a partir de estudos a colocação feita em "Machine learning methods to detect money laundering in theBitcoin blockchain in the presence of label scarcity" sobre a baixa eficiencia de modelos de anomaly detection nesse problema, ao utilizarmos o score de anomalia produzido por uma Isolation Forest (sendo um dos principais algoritmos dessa área) e observando uma visualização dos dados a partir dessa lente que apresenta diferenciação consideravelmente menor do que a obtida no uso de uma lente como o primeiro componente de um PLS, com seus componentes voltados a maximização da variancia linear em X que produza maior variancia em Y

In [ ]:
model = ensemble.IsolationForest(random_state= 42)
model.fit(X_train)
lens1 = model.decision_function(X_train).reshape((X_train.shape[0], 1))

lens2 = mapper.fit_transform(X_train, projection = "l2norm")


# Combine lenses pairwise to get a 2-D lens i.e. [Isolation Forest, L^2-Norm] lens
lens = np.c_[lens1, lens2]

In [ ]:
'''
graph = mapper.map(lens1,
                   X_train,
                   cover=km.Cover(n_cubes=15, perc_overlap=0.5),
                   clusterer=sklearn.cluster.KMeans(n_clusters = 2,
                                                    random_state= 42))

# Visualization
grafico = mapper.visualize(graph,
                 path_html="/kaggle/working/elliptic-dataset_EDA.html",
                 title=" Illicit Transaction - Elliptic Dataset - EDA with Heatmap True Label',
                 color_function= np.array(y_train))
'''

# 1) Otimização bayesiana de modelos state-of-art (wrapper + hyperparametros)


*Obs1: Busca utilizando cross-validation temporal com 4 folds (20 time_steps para treino - pula 4 - todo restante até time_step 34 para teste), avançando treino e teste um time_step para cada fold*

*Obs2: Uma busca de hyperparametros ótimos se torna enviesada por features que já tenham sido definidas; assim como uma busca por melhores features, sem variação de hiperparametros enviesa quais features serão mais propensas a serem escolhidas.*

In [ ]:
from hyperopt import hp
from hyperopt.pyll.stochastic import sample


{feature: hp.choice('feature_activation', [0, 1]) for feature in X_train.columns}


def wrapper(df, features = None):

    if features == None:
        return df
    
    else:
        active_features = []
        for col in features.keys():
            if features[col] == 1:
                active_features.append(col)
            
    return df[set(active_features + ['time_step'])]

import csv
from hyperopt import STATUS_OK
from timeit import default_timer as timer
from hyperopt import Trials


# Keep track of results
bayes_trials = Trials()

out_file = '/kaggle/working/hyperopt_trials_random_forest.csv'
of_connection = open(out_file, 'w')
writer = csv.writer(of_connection)

# Write the headers to the file
writer.writerow(['loss', 'params', 'iteration', 'estimators', 'train_time'])
of_connection.close()




####################################################################################################################



def objective(params):
    """Objective function for Hyperparameter Optimization"""
    
    # Keep track of evals
    global ITERATION
    
    ITERATION += 1

    
    start = timer()
    
    # Calls the estimator
    
    for parameter_name in ['max_depth']:
        params['hyperparams'][parameter_name] = int(params['hyperparams'][parameter_name])
    

    
    wrapped_df = wrapper(X_train, params['feature_activation'])
        
        
    # Declare classifier
    
    classifier = RandomForestClassifier(**params['hyperparams'])
    
    # Run the fit
    
    average_auc = []
    for i in range(0,4):
        cross_validation_train = (wrapped_df.join(y_train)).query(str(i) + ' < time_step <= ' + str(i+20))
        cross_validation_test = (wrapped_df.join(y_train)).query('time_step >= ' + str(i + 24))
        classifier.fit(cross_validation_train.drop(['target'], axis = 1), cross_validation_train['target'])
    
        y_score = classifier.predict_proba(cross_validation_test.drop(['target'], axis = 1))
        y_score = pd.DataFrame( y_score )[1]
        
        average_auc.append( roc_auc_score(cross_validation_test['target'], y_score) )
    
    
    # loss function
    loss = - np.array(average_auc).mean()
    
    
    
  ##############################################################################################################   
    
    
    print(ITERATION)
    
    #registers the time took
    run_time = timer() - start
    
    # Write to the csv file ('a' means append)
    of_connection = open(out_file, 'a')
    writer = csv.writer(of_connection)
    writer.writerow([loss, params, ITERATION, run_time])
    
    # Dictionary with information for evaluation
    return {'loss': loss, 'params': params, 'iteration': ITERATION,
            'train_time': run_time, 'status': STATUS_OK}





###################################################################################################################
from hyperopt import hp
from hyperopt.pyll.stochastic import sample

# Define the search space
space = {
    
    'feature_activation' : {feature : hp.choice(feature, [0, 1]) for feature in X_train.columns},
    
    'hyperparams' :  {
                    'n_estimators' : 75,
                    'max_samples' : hp.uniform('max_samples', 0.1, 1),
                    'max_features' : hp.uniform('max_features', 0.05, 1),
                    'max_depth' : hp.quniform('max_depth', 9, 35, 1),
                    'n_jobs' : -1
                     }
        }
    
params = sample(space)
params



##################################################################################################################




from hyperopt import tpe
# optimization algorithm
tpe_algorithm = tpe.suggest




###################################################################################################################



from hyperopt import fmin


from hyperopt import Trials
# Keep track of results
bayes_trials = Trials()


%%capture

# Global variable
global  ITERATION

ITERATION = 0

# Run optimization
best = fmin(fn = objective, space = space, algo = tpe.suggest, 
            max_evals = 1500, trials = bayes_trials, rstate = np.random.RandomState(50))

bayes_trials_results = sorted(bayes_trials.results, key = lambda x: x['loss'])
bayes_trials_results

* Parametrização abaixo escolhida a partir de 500 interações, sendo a mais simples obtida nos 5 primeiros resultados
* -0.848119159778934 roc_auc de teste na cross-validation - a mesma métrica avaliada no treino fica ao redor de 1.0, fornecendo diagnostico de muita variância

In [ ]:
parametrizacao = {'feature_activation': {'-0.0031749902209437055': 0, '-0.004094090363360379': 0, '-0.004194370240563965': 0, '-0.004357614038573012': 0, '-0.0106685610738475': 1, '-0.010763009512837094': 1, '-0.012005182118417812': 0, '-0.012107451777478418': 0, '-0.013281614870058885': 1, '-0.014658775691506424': 0, '-0.015070900001830685': 0, '-0.017031675880349647': 0, '-0.01764011962990492': 0, '-0.018848712954381765': 1, '-0.0230451563960962': 1, '-0.02466883065625352': 0, '-0.02605968882058813': 1, '-0.026214655177430907': 0, '-0.027659714028485525': 0, '-0.028741285856664783': 0, '-0.030026235277791913': 1, '-0.031272390486630317': 1, '-0.035390552600813516': 1, '-0.039143978122140564': 0, '-0.03914869407758738': 0, '-0.042955299258028254': 1, '-0.04387454791734898': 0, '-0.04970696439403985': 0, '-0.057194633660791555': 0, '-0.05901305478547111': 1, '-0.06158379407303222': 1, '-0.07525553154622772': 0, '-0.08014673584659182': 0, '-0.08014726965335221': 1, '-0.08345878401951694': 0, '-0.0874901561101501': 0, '-0.08879772536984745': 0, '-0.0904370818830761': 1, '-0.09314472701426248': 0, '-0.09540268927284089': 1, '-0.09752359377152515': 1, '-0.09771859912616487': 1, '-0.0978946778840727': 0, '-0.11300200928476244': 0, '-0.11642460389073964': 0, '-0.1168167152870516': 0, '-0.11979245961251665': 1, '-0.12061340670311574': 1, '-0.1209504155034722': 1, '-0.12196959975910057': 0, '-0.12746225048300552': 1, '-0.13115530389558736': 0, '-0.13454647495572797': 1, '-0.13732327709079195': 1, '-0.13973120192279553': 0, '-0.13973300164963834': 0, '-0.14033462388732307': 1, '-0.14076322246003314': 1, '-0.14370672448675942': 0, '-0.14890718609704007': 0, '-0.1489118870463073': 1, '-0.15246437130048263': 0, '-0.1556613943053229': 1, '-0.15566142432803598': 1, '-0.15973245222462992': 0, '-0.16092533142876464': 1, '-0.16093221729544638': 0, '-0.16209679981659642': 1, '-0.16311462864045503': 0, '-0.16312596586207964': 0, '-0.16440217329951': 1, '-0.16654994856468994': 0, '-0.16793302645225652': 0, '-0.1691595024198962': 1, '-0.16960915015560768': 1, '-0.17115370708833458': 0, '-0.1714692896288031': 0, '-0.17160149344087147': 0, '-0.1728840699195216': 0, '-0.17289511037923946': 1, '-0.1744725474413385': 1, '-0.1766172824237854': 0, '-0.18381594178200059': 1, '-0.18466755143291433': 0, '-0.18799267262971858': 1, '-0.19147194592540379': 1, '-0.19481661068542533': 1, '-0.19914489882884956': 1, '-0.20158367565021204': 0, '-0.21294750674116758': 1, '-0.21653637857822206': 0, '-0.21681436061843226': 1, '-0.22720332382768585': 1, '-0.22721544644782224': 1, '-0.23255262057859216': 1, '-0.23495151756654098': 0, '-0.2358964234692872': 0, '-0.23936836871437037': 1, '-0.24323609254239772': 1, '-0.2505232075535023': 1, '-0.25511064604426475': 1, '-0.2591940456978486': 0, '-0.2623679155254754': 0, '-0.2637032528675492': 0, '-0.26437568843944076': 0, '-0.29377256255240464': 0, '-0.29797501416794375': 1, '-0.3499327749819623': 0, '-0.37571453141267186': 1, '-0.4140054550165004': 0, '-0.4235879631572251': 1, '-0.44088282778065335': 1, '-0.4581618412195745': 1, '-0.46755439410398614': 0, '-0.48834041329936734': 1, '-0.5621534802884299': 1, '-0.6009988905192808': 1, '-0.6056310578523486': 1, '-0.7683292540299814': 1, '-0.968902874660573': 0, '-1.0159633735173061': 0, '-1.0162304272806977': 1, '-1.0963356939258175': 0, '-1.2013688016765636': 0, '-1.230440801051298': 1, '-1.2673399140326196': 0, '-1.315375381290753': 1, '-1.3153883882753692': 1, '-1.3163333780274464': 0, '-1.316342283664295': 1, '-1.3547347717590499': 0, '-1.3714598276027399': 1, '-1.3736571773938961': 0, '-1.3737824923785267': 0, '-1.4036981050621322': 1, '-1.4579532008792007': 1, '-1.4859719767717499': 0, '-1.494056831372085': 1, '0.001427813709709475': 1, '0.0014826437872997916': 0, '0.0024263098865888125': 0, '0.0031432964315106413': 0, '0.01827940003744589': 0, '0.034038660711490365': 1, '0.0365766503766988': 1, '0.0374680287743863': 0, '0.04234513480691014': 0, '0.043444221333941775': 1, '0.048766817854885264': 1, '0.05295606452358398': 0, '0.759748200156594': 0, '1.0637874876466933': 0, '1.0642045463479908': 1, '1.1255896087380572': 0, '1.1280380290020882': 0, '1.1335266830129689': 0, '1.135278702458467': 1, '1.135522751490673': 0, '1.1359467007331425': 0, '1.3407327277377104': 1, '1.342003002186918': 0, '1.4613303209554889': 1, '1.4613689382001922': 0, '1.487932476808418': 0, '1.4881129012284324': 0, 'time_step': 1}, 'hyperparams': {'max_depth': 11, 'max_features': 0.48930003686115775, 'max_samples': 0.7692800266308553, 'n_estimators': 250, 'n_jobs': -1}}

parametrizacao['hyperparams']['n_estimators'] = 300

colunas_escolhidas = list(wrapper(X_train, parametrizacao['feature_activation']).columns)

# 2) Aprendizagem semi-supervisionada

Devemos espremer o máximo possivel do cenario do problema - em que pouquissimas transações estão classificadas entre a população total

Os artigos existentes sobre o dataset elliptic disponiveis na internet se focam exclusivamente em seu estudo por uma abordagem supervisionada ou de anomaly detection, não utilizando técnicas de semi-supervisined learning, como forma de aumentar o dataset de treino a partir de transações já disponiveis, porem não classificadas inicialmente, e diminuir o problema de variância diagnosticado.



In [ ]:
rf = RandomForestClassifier(**parametrizacao['hyperparams'], random_state = 1)
sampled_df = X_train[colunas_escolhidas].sample(frac = 0.97, random_state = 1)
y_sampled = y_train.loc[sampled_df.index]
rf.fit( sampled_df, y_sampled )
y_labeled_score1 = rf.predict_proba( unlabeled_df_train[colunas_escolhidas] )
y_labeled_score1 = pd.DataFrame( y_labeled_score1, index = unlabeled_df_train.index ).rename(columns = {1 : 'y_labeled_score1'})
y_label1 = rf.predict( unlabeled_df_train[colunas_escolhidas] )
y_label1 = pd.DataFrame( y_label1, index = unlabeled_df_train.index ).rename(columns = {0 : 'y_label1'})
labels_1 = y_labeled_score1.join(y_label1)[['y_labeled_score1', 'y_label1']]


rf = RandomForestClassifier(**parametrizacao['hyperparams'], random_state = 2)
sampled_df = X_train[colunas_escolhidas].sample(frac = 0.97, random_state = 2)
y_sampled = y_train.loc[sampled_df.index]
rf.fit( sampled_df, y_sampled )
y_labeled_score2 = rf.predict_proba( unlabeled_df_train[colunas_escolhidas] )
y_labeled_score2 = pd.DataFrame( y_labeled_score2, index = unlabeled_df_train.index ).rename(columns = {1 : 'y_labeled_score2'})
y_label2 = rf.predict( unlabeled_df_train[colunas_escolhidas] )
y_label2 = pd.DataFrame( y_label2, index = unlabeled_df_train.index ).rename(columns = {0 : 'y_label2'})
labels_2 = y_labeled_score2.join(y_label2)[['y_labeled_score2', 'y_label2']]


rf = RandomForestClassifier(**parametrizacao['hyperparams'], random_state = 3)
sampled_df = X_train[colunas_escolhidas].sample(frac = 0.97, random_state = 3)
y_sampled = y_train.loc[sampled_df.index]
rf.fit( sampled_df, y_sampled )
y_labeled_score3 = rf.predict_proba( unlabeled_df_train[colunas_escolhidas] )
y_labeled_score3 = pd.DataFrame( y_labeled_score3, index = unlabeled_df_train.index ).rename(columns = {1 : 'y_labeled_score3'})
y_label3 = rf.predict( unlabeled_df_train[colunas_escolhidas] )
y_label3 = pd.DataFrame( y_label3, index = unlabeled_df_train.index ).rename(columns = {0 : 'y_label3'})
labels_3 = y_labeled_score3.join(y_label3)[['y_labeled_score3', 'y_label3']]

In [ ]:
labels_df = labels_1.join(labels_2).join(labels_3)
labels_df['target'] = -1
labels_df.loc[ labels_df.query('y_labeled_score1 > 0.8 and y_labeled_score2 > 0.8 and y_labeled_score3 > 0.8').index, ['target'] ] = 1
labels_df.loc[ labels_df.query('y_labeled_score1 <= 0.1 and y_labeled_score2 <= 0.1 and y_labeled_score3 <= 0.1').index, ['target'] ] = 0

unlabeled_df_train['target'] = labels_df['target']

# Primeira fase de resultado : Modelo State-of-art
1. Random Forest

* Performance base com hiperparametros e uso de 78 features das 166 ficou ao redor de 0.912 - 0.913

In [ ]:
rf = RandomForestClassifier(**parametrizacao['hyperparams'], random_state = 10)
rf.fit(X_train[colunas_escolhidas], y_train)

y_score = rf.predict_proba(X_test[colunas_escolhidas])
y_score = pd.DataFrame( y_score )[1]

roc_auc_score(y_test, y_score)


* Performance com dataset de treino expandido adicionado ficou ao redor de 0.920 - 0.921

In [ ]:
rf = RandomForestClassifier(**parametrizacao['hyperparams'], random_state = 1)
rf.fit(X_train[colunas_escolhidas].append(unlabeled_df_train.query('target != -1')[colunas_escolhidas]), y_train.append(unlabeled_df_train.query('target != -1')['target']))

y_score = rf.predict_proba(X_test[colunas_escolhidas])
y_score = pd.DataFrame( y_score, index = X_test.index )[1].rename('y_score')

roc_auc_score(y_test, y_score)

# Trust Score - test dataset

Medida proposta inicialmente nos artigos "Identifying unreliable predictions in clinical risk models" e "To trust or not a classifier", desenvolvida na área da saude para quantificar a confiabilidade que deveria ser dada as previsões de um modelo para cada caso específico; dado que podem existir subpopulações que o modelo erra sistematicamente, embora performe bem globalmente, com erros de tipo 1 ou tipo 2 tendo muito peso devido a suas implicações médicas

A nossa hipotese, discutida abaixo foi sua extenção para área de previsão de transações ilicitas; em que erros de tipo I e tipo II tambem tem graves consequencias para os individuos envolvidos. Pensamos em 3 aplicações possiveis dessa métrica
1. A diretamente proposta pelos artigos, de medida de confiabilidade das previsões do modelo para novas previsões realizadas (que assume importancia grande, dado estarmos em um cenario semi-supervisionado, em que as previsões podem ter implicações grandes e não temos a classificação direta sobre a maior parte dos dados)
2. A de possivel filtro a ser aplicado nas observações de um modelo semi-supervisionado - em que observações que tivessem trust scores muito baixos seriam cortadas, de maneira aos exemplos inicialmente não classificados e com baixa confiabilidade sobre as previsões não serem utilizados na etapa de retreino do modelo, na qual utiliza dados classificados por ele mesmo em seu conjunto de treino
3. Como medida possivel sobre performance do modelo pelo tempo; dado o cenário de que podem existir meses com pouquissimas observações classificadas, mas ainda sim podendo ter uma métrica que oriente noções gerais sobre a sua estabilidade no tempo

In [ ]:
try:
    X_train.drop(['erro_tipo_1', 'erro_tipo_2', 'lda_transform_erro_1_test', 'lda_transform_erro_2_test'], axis = 1, inplace = True)
    X_test.drop(['erro_tipo_1', 'erro_tipo_2', 'lda_transform_erro_1_test', 'lda_transform_erro_2_test'], axis = 1, inplace = True)
except:
    None



from sklearn.preprocessing import Normalizer


norm = Normalizer().fit(X_train)  # fit does nothing.

X_train_norm = pd.DataFrame( norm.transform(X_train), index = X_train.index)


lof = LocalOutlierFactor(n_neighbors = 10, n_jobs = -1, novelty = False)

y_outlier = lof.fit_predict(X_train_norm)


###############################################################3


from sklearn.cross_decomposition import PLSRegression

pls = PLSRegression(n_components= 15)
pls.fit(X_train, y_train)


X_train_transformed = pd.DataFrame( pls.transform(X_train), index = X_train.index)

In [ ]:
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import StratifiedShuffleSplit

sss = StratifiedShuffleSplit(n_splits = 5, test_size=0.333, random_state=0)
sss.get_n_splits(X_train_transformed, y_train)

trust_score_final = []
trust_score_df = (X_train_transformed.join(y_train).reset_index()).join(pd.Series (y_outlier).rename('outlier_treino') )
for train_index, test_index in sss.split(X_train_transformed, y_train):

    searched_df = trust_score_df.iloc[train_index].query('outlier_treino != -1')
    searched_df = searched_df.drop(['index', 'outlier_treino'], axis = 1)
    
    searched_df_target0 = searched_df.query(y_train.name + " == 0")
    neigh_target0 = NearestNeighbors(n_neighbors = 2, n_jobs = -1)
    neigh_target0.fit(searched_df_target0.drop([y_train.name], axis =1))

    searched_df_target1 = searched_df.query(y_train.name + " == 1")
    neigh_target1 = NearestNeighbors(n_neighbors = 2, n_jobs = -1)
    neigh_target1.fit(searched_df_target1.drop([y_train.name], axis =1))

    # Busca 2 n_neighbours, de forma a retornar pela funcao abaixo o valor minimo diferente de zero,
    # caso exista duplicata de elemento de X_test em X_train, em que distancia seria 0

    distance_class_0 = list(map( lambda x: np.min(x[np.nonzero(x)]), neigh_target0.kneighbors(pls.transform(X_test))[0]))

    distance_class_1 = list(map( lambda x: np.min(x[np.nonzero(x)]), neigh_target1.kneighbors(pls.transform(X_test))[0]))

    # trust score = distancia para classe diferente da prevista / distancia para classe prevista

    trust_score = []
    for i in range (0, len(y_test)):
        trust_score.append(  distance_class_0[i] / distance_class_1[i] if y_score.tolist()[i] > 0.5 else
                             distance_class_1[i] / distance_class_0[i])

    trust_score = list(map( float, trust_score ))
    trust_score_final.append(trust_score)
trust_score_final_df = pd.DataFrame( trust_score_final, columns = X_test.index ).mean().rename('trust_score')

# Reflexões sobre Trust Score

* A hipótese inicial, pensada a partir dos artigos "Identifying unreliable predictions in clinical risk models" e "To trust or not a classifier"  era de que o comportamento do score de confiabilidade teria uma relação linear, ou ao menos monotonica, com a imprecisão medida nas probabilidades geradas pelos modelos - com os menores valores concentrando as previsões menos confiaveis

* Essa hipotese estava equivocada - impedindo obtenção de melhora de performance no modelo pelo uso do trust score como um filtro das classificações para retreino do modelo semi-supervisionado - mas nessa étapa - embora tenhamos posteriormente descobertos que existe uma relação não linear, indicada abaixo

* A média e desvio padrão dos Trust Scores de previsões certas e erradas são sensivelmente diferentes, com o trust score de previsões erradas sendo bastante mais concentrado e tendo desvio padrão 3 vezes menor frente ao desvio padrão das acertadas

# Média e desvio padrão do trust score das classificações erradas - test set

In [ ]:
query_erros = '(target == 1 and y_score < 0.5) or (target == 0 and y_score > 0.5)'


print('A quantidade de previsões erradas é : ' + str( len(X_test.join(y_test).join(trust_score_final_df).join(y_score).query( query_erros )) ) )
print('Média de trust_score em são previsões erradas : ' + str(X_test.join(y_test).join(trust_score_final_df).join(y_score).query( query_erros )[['trust_score']].mean()[0]) )
print('Desvio padrão do Trust Score em previsões erradas : ' + str(X_test.join(y_test).join(trust_score_final_df).join(y_score).query( query_erros )[['trust_score']].std()[0]) )

# Média e desvio padrão do trust score das classificações certas - test set

In [ ]:
query_acertos = '(y_score > 0.5 and target == 1) or (y_score < 0.5 and target == 0)'


print('A quantidade de previsões acertadas é : ' + str( len(X_test.join(y_test).join(trust_score_final_df).join(y_score).query( query_acertos )) ) )
print('Média de trust_score em são previsões certas : ' + str(X_test.join(y_test).join(trust_score_final_df).join(y_score).query( query_acertos )[['trust_score']].mean()[0]) )
print('Desvio padrão do Trust Score em previsões certas : ' + str(X_test.join(y_test).join(trust_score_final_df).join(y_score).query( query_acertos )[['trust_score']].std()[0]) )

* A checagem abaixo sobre intervalos gerados pela média e desvio padrão do Trust Score revela uma informação interessante
* A partir dessas duas informações, é possivel gerar um intervalo com propriedade interessante pelos dados com Trust Score dentro dele -  em um intervalo que contem 25% (4000 observações) de todo dataset de teste, estão concentradas aproximamente de 50% de todos erros de previsão cometidos pelo modelo; enquanto os outros 75% (12000 observações) representam aproximadamente os 50% restantes - informação interessante

In [ ]:
# Query que seleciona todas classificacoes erradas
query1_inner = '(target == 1 and y_score < 0.5) or (target == 0 and y_score > 0.5) '

centro_intervalo_trust_score_erros = X_test.join(y_test).join(trust_score_final_df).join(y_score).query( query1_inner )[['trust_score']].mean()[0] - 0.05
amplitude_trust_score_erros = X_test.join(y_test).join(trust_score_final_df).join(y_score).query( query1_inner )[['trust_score']].std()[0] - 0.1

query2_inner = ' (trust_score > ' + str(centro_intervalo_trust_score_erros - amplitude_trust_score_erros) + ' and trust_score < ' + str(centro_intervalo_trust_score_erros + amplitude_trust_score_erros) + ')'

print('Selecao de todas classificacoes erradas em intervalo definido : ' + str([centro_intervalo_trust_score_erros - amplitude_trust_score_erros, centro_intervalo_trust_score_erros + amplitude_trust_score_erros]))
print('Quantidade de erros nesse intervalo : ' + str(len(X_test.join(y_test).join(trust_score_final_df).join(y_score).query(query1_inner + ' and ' + query2_inner)[['y_score', 'target', 'trust_score']])))
print('Quantidade de transações nesse intervalo : ' + str(len(X_test.join(y_test).join(trust_score_final_df).join(y_score).query(query2_inner))))

In [ ]:
query1_outer = '(target == 1 and y_score < 0.5) or (target == 0 and y_score > 0.5) '
query2_outer = ' (trust_score < ' + str(centro_intervalo_trust_score_erros - amplitude_trust_score_erros) + ' or trust_score > ' + str(centro_intervalo_trust_score_erros + amplitude_trust_score_erros) + ')'

print('Selecao de todas classificacoes erradas disjuntas ao intervalo acima')
print('Quantidade de erros nesse intervalo : ' + str(len(X_test.join(y_test).join(trust_score_final_df).join(y_score).query(query1_outer + ' and ' + query2_outer)[['y_score', 'target', 'trust_score']])))
print('Quantidade de transações nesse intervalo : ' + str(len(X_test.join(y_test).join(trust_score_final_df).join(y_score).query(query2_outer))))

# Essas informações indicam que a possibilidade de estabelecer um intervalo de valores a partir do trust score medido para cada amostra, na qual se concentram os erros de previsão realizados


* Como tentativa da verificação sobre a possivel estabilidade temporal nessa hipotese, realizamos mais um experimento; em que o modelo foi retreinado apenas nos time_steps 1 até 23 e mesma analise foi realizada com a separação para teste dos steps de 28 até 34
* A confirmação segue abaixo, em que tambem nesses meses, mesmo com modelo sendo treinado em menor quantidade de dados e tendo menor performance, é indicada a existencia de estabilidade temporal nessa informação, dado que os resultados se mantem ao redor de um desvio padrão de distancia dos resultados obtidos anteriormente, assim como tendo um desvio padrão consideravelmente baixo frente aos desvios padrões obtidos pelas transações acertadas

In [ ]:
try:
    X_train.drop(['erro_tipo_1', 'erro_tipo_2', 'lda_transform_erro_1_test', 'lda_transform_erro_2_test'], axis = 1, inplace = True)
    X_test.drop(['erro_tipo_1', 'erro_tipo_2', 'lda_transform_erro_1_test', 'lda_transform_erro_2_test'], axis = 1, inplace = True)
except:
    None


from sklearn.preprocessing import Normalizer

X_train_reduced = X_train.query('time_step <= 23')[colunas_escolhidas]
y_train_reduced = y_train.loc[X_train_reduced.index]
X_new_test = X_train.query('time_step >= 28')[colunas_escolhidas]
y_new_test = y_train.loc[X_new_test.index]


rf.fit(X_train_reduced, y_train_reduced)
y_predicted = rf.predict_proba(X_new_test)
y_predicted = pd.DataFrame( y_predicted, index = X_new_test.index)[1].rename('y_predicted')





norm = Normalizer().fit(X_train_reduced)  # fit does nothing.

X_train_norm = pd.DataFrame( norm.transform(X_train_reduced), index = X_train_reduced.index)


lof = LocalOutlierFactor(n_neighbors = 10, n_jobs = -1, novelty = False)

y_outlier = lof.fit_predict(X_train_norm)


###############################################################3


from sklearn.cross_decomposition import PLSRegression

pls = PLSRegression(n_components= 15)
pls.fit(X_train_reduced, y_train_reduced)





X_train_transformed = pd.DataFrame( pls.transform(X_train_reduced), index = X_train_reduced.index)

In [ ]:
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import StratifiedShuffleSplit

sss = StratifiedShuffleSplit(n_splits = 5, test_size=0.333, random_state=0)
sss.get_n_splits(X_train_transformed, y_train_reduced)

trust_score_final = []
trust_score_df = (X_train_transformed.join(y_train_reduced).reset_index()).join(pd.Series (y_outlier).rename('outlier_treino') )
for train_index, test_index in sss.split(X_train_transformed, y_train_reduced):

    searched_df = trust_score_df.iloc[train_index].query('outlier_treino != -1')
    searched_df = searched_df.drop(['index', 'outlier_treino'], axis = 1)
    
    searched_df_target0 = searched_df.query(y_train_reduced.name + " == 0")
    neigh_target0 = NearestNeighbors(n_neighbors = 2, n_jobs = -1)
    neigh_target0.fit(searched_df_target0.drop([y_train_reduced.name], axis =1))

    searched_df_target1 = searched_df.query(y_train_reduced.name + " == 1")
    neigh_target1 = NearestNeighbors(n_neighbors = 2, n_jobs = -1)
    neigh_target1.fit(searched_df_target1.drop([y_train_reduced.name], axis =1))

    # Busca 2 n_neighbours, de forma a retornar pela funcao abaixo o valor minimo diferente de zero,
    # caso exista duplicata de elemento de X_test em X_train, em que distancia seria 0

    distance_class_0 = list(map( lambda x: np.min(x[np.nonzero(x)]), neigh_target0.kneighbors(pls.transform(X_new_test))[0]))
    
    distance_class_1 = list(map( lambda x: np.min(x[np.nonzero(x)]), neigh_target1.kneighbors(pls.transform(X_new_test))[0]))

    # trust score = distancia para classe diferente da prevista / distancia para classe prevista

    trust_score = []
    for i in range (0, len(y_new_test)):
        trust_score.append(  distance_class_0[i] / distance_class_1[i] if y_predicted.tolist()[i] > 0.5 else
                             distance_class_1[i] / distance_class_0[i])

    trust_score = list(map( float, trust_score ))
    trust_score_final.append(trust_score)
trust_score_final_df = pd.DataFrame( trust_score_final, columns = X_new_test.index ).mean().rename('trust_score')

In [ ]:
query_acertos = '(y_predicted > 0.5 and target == 1) or (y_predicted < 0.5 and target == 0)'

print('A quantidade de previsões acertadas é : ' + str( len(X_new_test.join(y_new_test).join(trust_score_final_df).join(y_predicted).query( query_acertos )) ) )
print('Média de trust_score em são previsões certas : ' + str( X_new_test.join(y_new_test).join(trust_score_final_df).join(y_predicted).query( query_acertos )['trust_score'].mean()) )
print('Desvio padrão do Trust Score em previsões certas : ' + str(X_new_test.join(y_new_test).join(trust_score_final_df).join(y_predicted).query( query_acertos )['trust_score'].std()) )

In [ ]:
query_erros = '(y_predicted > 0.5 and target == 0) or (y_predicted < 0.5 and target == 1)'

print('A quantidade de previsões erradas é : ' + str( len( X_new_test.join(y_new_test).join(trust_score_final_df).join(y_predicted).query( query_erros ) ) ))
print('Média de trust_score em são previsões equivocadas : ' + str( X_new_test.join(y_new_test).join(trust_score_final_df).join(y_predicted).query( query_erros )['trust_score'].mean() ))
print('Desvio padrão do Trust Score em previsões equivocadas : ' + str( X_new_test.join(y_new_test).join(trust_score_final_df).join(y_predicted).query( query_erros )['trust_score'].std()) )

* Essa segunda testagem, em outro periodo de tempo, sugere a existencia de estabilidade temporal na média e desvio padrão desse padrão observado no trust scores das classificações erradas, com esses valores tendo sido obtidos por uma segunda vez muito próximos dos obtidos inicialmente, embora com testagem feita em outro conjunto de meses - com um intervalo gerado a partir desses valores podendo ser usado para indicação de confiabilidade das classificações

# 3) Lentes para projeção de forma a explicitar erros sistemáticos do modelo na visualização

* Abaixo segue o terceiro passo planejado; a visualização dos dados a partir de lentes que explicitem as zonas de erro e agrupem dados classificados erroneamente de maneira semelhante
* Para isso, utilizamos quatro funções combinadas para gerar uma lente; a probabilidade produzida para a verdadeira classe de cada transação, a probabilidade produzida para a classe prevista da transação, a classe real das transações (para separar claramente o comportamento nas duas classes) e o primeiro componente de um PCA (para obtenção de formas mais detalhadas)


In [ ]:
from sklearn.preprocessing import Normalizer

norm = Normalizer().fit(X_train)  # fit does nothing.

X_train_norm = pd.DataFrame( norm.transform(X_train), index = X_train.index)

In [ ]:
predicted_score_true_label = []

predicted_score_predicted_label = []
for i in range(0 , len(y_score)):
    predicted_score_true_label.append( y_score.tolist()[i] if y_test.tolist()[i] == 1 else 1 - y_score.tolist()[i] )
    predicted_score_predicted_label.append( max([y_score.tolist()[i], 1 - y_score.tolist()[i]]) )


from sklearn.decomposition import PCA

pca = PCA(n_components = 1)
pca.fit(X_train)
lens_pca = pca.transform(X_test)

lens = np.c_[lens_pca, y_test, predicted_score_true_label, predicted_score_predicted_label]

In [ ]:
graph = mapper.map(lens,
                   norm.transform(X_test),
                   cover=km.Cover(n_cubes= 5, perc_overlap=0.5),
                   remove_duplicate_nodes = True,
                   clusterer=sklearn.cluster.KMeans(n_clusters=2,
                                                    random_state= 42))


# Visualization
grafico = mapper.visualize(graph,
                 path_html="/kaggle/working/elliptic-dataset  Erro Analysis Prediction Score.html",
                 title=" Illicit Transaction - Elliptic Dataset - Error Analysis with Heatmap Prediction Score",
                 color_function= np.array(y_score))


# Visualization
grafico = mapper.visualize(graph,
                 path_html="/kaggle/working/elliptic-dataset Error Analysis True Label.html",
                 title=" Illicit Transaction - Elliptic Dataset -  Error Analysis with Heatmap True Label",
                 color_function= np.array(y_test))

# Acima se encontra a primeira visualização obtida do dataset de teste - uma contendo heatmap a partir da classe real e outra contendo heatmap a partir da probabilidade gerada de ser ilicita pelo modelo

* Abaixo, as principais zonas que contem os principais erros de tipo 1 e tipo 2 serão agrupadas a partir de um algoritmo de community detection em grafos
* Em seguida, uso os index dos membros que pertencem a essas zonas para criar duas novas features a serem adicionadas no modelo - voltadas especificamente para a Random Forest consegua ter acesso a váriaveis que indiquem quando as previsões que esta realizando são semelhantes a erros cometidos; assim podendo os corrigir

In [ ]:

from sklearn.preprocessing import Normalizer

X_train_reduced = X_train.query('time_step <= 23')[colunas_escolhidas]
y_train_reduced = y_train.loc[X_train_reduced.index]
X_new_test = X_train.query('time_step >= 28')[colunas_escolhidas]
y_new_test = y_train.loc[X_new_test.index]


rf.fit(X_train_reduced, y_train_reduced)
y_predicted = rf.predict_proba(X_new_test)
y_predicted = pd.DataFrame( y_predicted, index = X_new_test.index)[1].rename('y_predicted')


from sklearn.preprocessing import Normalizer

norm = Normalizer().fit(X_train)  # fit does nothing.

X_train_norm = pd.DataFrame( norm.transform(X_train), index = X_train.index)

In [ ]:
predicted_score_true_label = []

predicted_score_predicted_label = []
for i in range(0 , len(y_predicted)):
    predicted_score_true_label.append( y_predicted.tolist()[i] if y_new_test.tolist()[i] == 1 else 1 - y_predicted.tolist()[i] )
    predicted_score_predicted_label.append( max([y_predicted.tolist()[i], 1 - y_predicted.tolist()[i]]) )


from sklearn.decomposition import PCA

pca = PCA(n_components = 1)
pca.fit(X_train_reduced)
lens_pca = pca.transform(X_new_test)

lens = np.c_[lens_pca, y_new_test, predicted_score_true_label, predicted_score_predicted_label]

In [ ]:
graph = mapper.map(lens,
                   norm.transform(X_new_test),
                   cover=km.Cover(n_cubes= 5, perc_overlap=0.5),
                   remove_duplicate_nodes = True,
                   clusterer=sklearn.cluster.KMeans(n_clusters=2,
                                                    random_state= 42))


# Visualization
grafico = mapper.visualize(graph,
                 path_html="/kaggle/working/elliptic-dataset  Erro Analysis Prediction Score new_test.html",
                 title=" Illicit Transaction - Elliptic Dataset - Heatmap Prediction Score new_test",
                 color_function= np.array(y_predicted))


# Visualization
grafico = mapper.visualize(graph,
                 path_html="/kaggle/working/elliptic-dataset Error Analysis True Label new_test.html",
                 title=" Illicit Transaction - Elliptic Dataset - Heatmap True Label new_test",
                 color_function= np.array(y_new_test))

In [ ]:
adapted_graph = km.adapter.to_networkx(graph)

from matplotlib.pyplot import figure
figure(figsize=(20, 10))


import community as community_louvain
import networkx as nx
import matplotlib.cm as cm

#G = nx.erdos_renyi_graph(100, 0.01)
partition = community_louvain.best_partition(adapted_graph, resolution = 1.3, random_state = 0)


# draw the graph
pos = nx.spring_layout(adapted_graph)
# color the nodes according to their partition
cmap = cm.get_cmap('viridis', max(partition.values()) + 1)
nx.draw_networkx_nodes(adapted_graph, pos, partition.keys(), node_size=40, 
                        cmap=cmap, node_color=list(partition.values()))
nx.draw_networkx_edges(adapted_graph, pos, alpha=0.5)

'''
Particao de erro tipo 1
Modelo indica ser classe 1 mas era classe 0
'''

lista = [] for i in partition.keys(): if partition[i] == 2: lista.append(i)

ids = [] for elemento in lista: if elemento in graph['nodes'].keys(): ids.append(graph['nodes'][elemento])

lista_ids_comunity_2 = list( set([item for sublist in ids for item in sublist]) ) print(lista_ids_comunity_2) len(lista_ids_comunity_2)


#########################################################################

'''
Particao de erro tipo 2
Modelo indica ser classe 0 mas era classe 1
'''

lista = [] for i in partition.keys(): if partition[i] == 0: lista.append(i)

ids = [] for elemento in lista: if elemento in graph['nodes'].keys(): ids.append(graph['nodes'][elemento])

lista_ids_comunity_0 = list( set([item for sublist in ids for item in sublist]) ) print(lista_ids_comunity_0) len(lista_ids_comunity_0)

In [ ]:
lista_ids_comunity_2 = [512, 1031, 523, 1036, 524, 2061, 1040, 528, 529, 1041, 1045, 1046, 535, 536, 1572, 3622, 1066, 1067, 4144, 4145, 570, 572, 578, 579, 1096, 585, 1614, 1117, 1119, 1120, 611, 612, 1123, 2148, 617, 3700, 2679, 2177, 2178, 645, 646, 1160, 140, 2188, 1168, 659, 155, 1182, 1695, 1696, 1697, 163, 164, 1190, 171, 1207, 1212, 2239, 2248, 1738, 4302, 4303, 723, 724, 1254, 1262, 1264, 1266, 3319, 252, 1279, 1285, 1801, 1802, 4881, 1303, 792, 1306, 794, 1308, 285, 1313, 1317, 1321, 298, 826, 1851, 2366, 1351, 841, 842, 1355, 1365, 1885, 863, 2404, 1384, 1385, 4458, 1387, 878, 1904, 371, 883, 884, 374, 887, 1403, 386, 900, 904, 2441, 1420, 1422, 920, 939, 4524, 2477, 433, 1972, 2488, 442, 3518, 1471, 1472, 456, 972, 461, 462, 470, 471, 1496, 2525, 993, 488, 489, 490, 2028, 2550, 2553]

lista_ids_comunity_0 = [4108, 4119, 4126, 2083, 4135, 2100, 4162, 2129, 4186, 4187, 2144, 2164, 2171, 128, 2183, 4235, 2200, 2204, 192, 4288, 2249, 4301, 209, 2258, 4315, 2269, 2271, 2277, 2280, 2284, 2296, 2319, 2322, 2339, 2342, 2345, 2369, 4419, 2373, 2375, 2383, 2386, 339, 2388, 347, 2406, 362, 2415, 4468, 4469, 2422, 2426, 2454, 2455, 4506, 2461, 4509, 4527, 4528, 2481, 2485, 2494, 4547, 4548, 4551, 4559, 2518, 2528, 4579, 4583, 2536, 4586, 2541, 2542, 4597, 4603, 4624, 538, 2591, 4645, 2604, 2611, 4662, 2625, 4674, 2629, 4680, 4681, 2634, 2635, 595, 596, 4705, 4718, 4729, 4759, 2717, 2720, 2721, 4776, 4790, 4796, 4802, 2757, 2758, 2773, 4824, 2780, 4848, 4854, 2818, 2828, 2829, 782, 2836, 2850, 4905, 2863, 2864, 2867, 2868, 2884, 2893, 2894, 2902, 2903, 4954, 2907, 2909, 4969, 2930, 2935, 2959, 2965, 2966, 2979, 2980, 2982, 2983, 2988, 2990, 943, 2991, 2999, 3003, 960, 964, 3012, 3015, 3016, 3020, 3025, 980, 3029, 1018, 3091, 3092, 3093, 3101, 3124, 3125, 3136, 3140, 1100, 3154, 3155, 3157, 3159, 3168, 3178, 3180, 3189, 3201, 3202, 3204, 3205, 3206, 3207, 3212, 3213, 3218, 3219, 3221, 1174, 3224, 3241, 3253, 3254, 3269, 3270, 1228, 3280, 3286, 3293, 3298, 3301, 3303, 3304, 3322, 3327, 3332, 3335, 3336, 3348, 3355, 3373, 3375, 3376, 1335, 3395, 3396, 3405, 3406, 3409, 3413, 1371, 3422, 3446, 3447, 3450, 3451, 3456, 3457, 3465, 3466, 3478, 3482, 1435, 3483, 3485, 3490, 3491, 3493, 3494, 3504, 1480, 3532, 3550, 3556, 1509, 1510, 3559, 3577, 3580, 3583, 3591, 3595, 3598, 3606, 3607, 1566, 1576, 3656, 3664, 3669, 3671, 3672, 3677, 3682, 3697, 3715, 3733, 3734, 3737, 3741, 3742, 3765, 3782, 3787, 3788, 3790, 3791, 3797, 3803, 3804, 3805, 3806, 3808, 3811, 3813, 3815, 3817, 3823, 3824, 3829, 3834, 3839, 3844, 3849, 3857, 3861, 3862, 3870, 1825, 3878, 3886, 3891, 3895, 3896, 3908, 3936, 3938, 3964, 1923, 1931, 3991, 3994, 1952, 1957, 4006, 1962, 1967, 4017, 1971, 4050, 2007, 4057, 4058, 2012, 2016, 4064, 4070, 4076, 2033, 2041]

In [ ]:
X_train['erro_tipo_1'] = 0
index_community_2 = list(X_train.iloc[lista_ids_comunity_2].index)
X_train.loc[index_community_2, ['erro_tipo_1']] = 1

X_train['erro_tipo_2'] = 0
index_community_0 = list(X_train.iloc[lista_ids_comunity_0].index)
X_train.loc[index_community_0, ['erro_tipo_2']] = 1

# Abaixo são geradas as duas variaveis novas - produzidas em dois passos simples:
1. A aplicação de um Partial Least Square no dataset, com formato one-versus-all, com target sendo todas transações que foram agrupadas na mesma comunidade com os principais erros de seu próprio tipo (seja tipo 1 ou tipo 2)
2. O uso da redução de dimensionalidade por um LDA, aplicado sobre as dimensões não correlacionadas geradas pela PLS, reduzindo efetivamente para uma dimensão para cada tipo de erro
3. Obtemos assim duas novas variaveis a serem adicionadas na Random Forest, no lugar da criação de uma nova camada de ensembles sobre as decisões feitas por ela, complexificando seus resultados


** Obs: os dois métodos aplicados (PLS e LDA) assumem pressupostos sobre os dados, que podem não valer; por isso, testamos todo o processo anterior com essas novas duas variaveis produzidas*

In [ ]:
try:
    X_train.drop(['lda_transform_erro_1_test', 'lda_transform_erro_2_test'], axis = 1, inplace = True)
except:
    None


from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

pls_erro_1 = PLSRegression(n_components= 50)
pls_erro_1.fit(X_train.drop(['erro_tipo_1', 'erro_tipo_2'], axis = 1), X_train['erro_tipo_1'])

lda_erro_1 = LinearDiscriminantAnalysis()
lda_transform_erro_tipo_1 = lda_erro_1.fit_transform(pls_erro_1.transform( X_train.drop(['erro_tipo_1', 'erro_tipo_2'], axis = 1) ), X_train['erro_tipo_1'])
lda_transform_erro_tipo_1 = pd.DataFrame(lda_transform_erro_tipo_1, index = X_train.index)[0].rename('lda_transform_erro_1_test')
lda_proba_erro_tipo_1 = pd.DataFrame( lda_erro_1.predict_proba(pls_erro_1.transform( X_train.drop(['erro_tipo_1', 'erro_tipo_2'], axis = 1) )) )[1].rename('lda_prob_erro_tipo_1')

#############################################################


pls_erro_2 = PLSRegression(n_components= 50)
pls_erro_2.fit(X_train.drop(['erro_tipo_1', 'erro_tipo_2'], axis = 1), X_train['erro_tipo_2'])

lda_erro_2 = LinearDiscriminantAnalysis()
lda_transform_erro_tipo_2 = lda_erro_2.fit_transform(pls_erro_2.transform( X_train.drop(['erro_tipo_1', 'erro_tipo_2'], axis = 1) ), X_train['erro_tipo_2'])
lda_transform_erro_tipo_2 = pd.DataFrame(lda_transform_erro_tipo_2, index = X_train.index)[0].rename('lda_transform_erro_2_test')
lda_proba_erro_tipo_2 = pd.DataFrame( lda_erro_1.predict_proba(pls_erro_1.transform( X_train.drop(['erro_tipo_1', 'erro_tipo_2'], axis = 1) )) )[1].rename('lda_prob_erro_tipo_2')


##############################################################

X_train['lda_transform_erro_1_test'] = lda_transform_erro_tipo_1
X_train['lda_transform_erro_2_test'] = lda_transform_erro_tipo_2

# Foi observado o impacto dos resultados abaixo com teste em diversas seeds diferentes para constatar variações aleatórias
* A nova variavel criada para os principais erros de tipo 1 retornou resultados consistentemente acima dos resultados sem ela, com AUC ficando entre 0.917 a 0.921 (comparavel com resultado obtido anteriormente em situação após retreino do modelo com expanção do dataset de treino com novas classificação não inicialmente classificadas)
* A nova variavel criada para os principais erros de tipo 2 não retornou resultados de melhora como a de tipo 1 - permanecendo na zona de AUC anteriora do modelo; e inclusive diminuindo a performance do modelo quando combinada com a variavel criada para identificação de erro de tipo 1 (provavelmente devido a overfitting de algum tipo ou variação temporal do comportamento desse erro) 

In [ ]:
rf = RandomForestClassifier(**parametrizacao['hyperparams'], random_state = 10)
rf.fit(X_train[colunas_escolhidas + ['lda_transform_erro_1_test']], y_train)


# Criacao das variaveis indicadoras de erro tipo 1 e 2 no teste

lda_transform_erro_1_test = lda_erro_1.transform(pls_erro_1.transform( X_test ) )
lda_transform_erro_1_test = pd.DataFrame( lda_transform_erro_1_test, index = X_test.index)[0].rename('lda_transform_erro_1_test')

#lda_transform_erro_2_test = lda_erro_2.transform(pls_erro_2.transform( X_test ) )
#lda_transform_erro_2_test = pd.DataFrame( lda_transform_erro_2_test, index = X_test.index)[0].rename('lda_transform_erro_2_test')

#lda_transform_erros = pd.DataFrame(lda_transform_erro_1_test).join(lda_transform_erro_2_test)

###################################

y_score = rf.predict_proba(X_test[colunas_escolhidas].join(lda_transform_erro_1_test))
y_score = pd.DataFrame( y_score )[1]

roc_auc_score(y_test, y_score)

In [ ]:
rf = RandomForestClassifier(**parametrizacao['hyperparams'], random_state = 10)
rf.fit(X_train[colunas_escolhidas + ['lda_transform_erro_2_test']], y_train)


# Criacao das variaveis indicadoras de erro tipo 1 e 2 no teste

#lda_transform_erro_1_test = lda_erro_1.transform(pls_erro_1.transform( X_test ) )
#lda_transform_erro_1_test = pd.DataFrame( lda_transform_erro_1_test, index = X_test.index)[0].rename('lda_transform_erro_1_test')

lda_transform_erro_2_test = lda_erro_2.transform(pls_erro_2.transform( X_test ) )
lda_transform_erro_2_test = pd.DataFrame( lda_transform_erro_2_test, index = X_test.index)[0].rename('lda_transform_erro_2_test')

lda_transform_erros = pd.DataFrame(lda_transform_erro_1_test).join(lda_transform_erro_2_test)

###################################

y_score = rf.predict_proba(X_test[colunas_escolhidas].join(lda_transform_erro_2_test))
y_score = pd.DataFrame( y_score )[1]

roc_auc_score(y_test, y_score)

# Com esses resultados, todo o processo anterior de aprendizado semi-supervisionado foi repetido - com adição de váriavel de identificação para erro tipo I

In [ ]:
lda_transform_erro_1_unlabeled = lda_erro_1.transform( pls_erro_1.transform( unlabeled_df_train.drop(['target'], axis = 1) ) ) 
lda_transform_erro_1_unlabeled = pd.DataFrame( lda_transform_erro_1_unlabeled, index = unlabeled_df_train.index)[0].rename('lda_transform_erro_1_test')

In [ ]:
try:
    unlabeled_df_train.drop(['target'], axis = 1, inplace = True)
except:
    None

lda_transform_erro_1_unlabeled = lda_erro_1.transform(pls_erro_1.transform( unlabeled_df_train ) ) 
lda_transform_erro_1_unlabeled = pd.DataFrame( lda_transform_erro_1_unlabeled, index = unlabeled_df_train.index)[0].rename('lda_transform_erro_1_test')

novas_colunas_escolhidas = colunas_escolhidas + ['lda_transform_erro_1_test']


rf = RandomForestClassifier(**parametrizacao['hyperparams'], random_state = 1)
sampled_df = X_train[novas_colunas_escolhidas].sample(frac = 0.97, random_state = 1)
y_sampled = y_train.loc[sampled_df.index]
rf.fit( sampled_df, y_sampled )
y_labeled_score1 = rf.predict_proba( unlabeled_df_train.join(lda_transform_erro_1_unlabeled)[novas_colunas_escolhidas] )
y_labeled_score1 = pd.DataFrame( y_labeled_score1, index = unlabeled_df_train.index ).rename(columns = {1 : 'y_labeled_score1'})
y_label1 = rf.predict( unlabeled_df_train.join(lda_transform_erro_1_unlabeled)[novas_colunas_escolhidas] )
y_label1 = pd.DataFrame( y_label1, index = unlabeled_df_train.index ).rename(columns = {0 : 'y_label1'})
labels_1 = y_labeled_score1.join(y_label1)[['y_labeled_score1', 'y_label1']]


rf = RandomForestClassifier(**parametrizacao['hyperparams'], random_state = 2)
sampled_df = X_train[novas_colunas_escolhidas].sample(frac = 0.97, random_state = 2)
y_sampled = y_train.loc[sampled_df.index]
rf.fit( sampled_df, y_sampled )
y_labeled_score2 = rf.predict_proba( unlabeled_df_train.join(lda_transform_erro_1_unlabeled)[novas_colunas_escolhidas] )
y_labeled_score2 = pd.DataFrame( y_labeled_score2, index = unlabeled_df_train.index ).rename(columns = {1 : 'y_labeled_score2'})
y_label2 = rf.predict( unlabeled_df_train.join(lda_transform_erro_1_unlabeled)[novas_colunas_escolhidas] )
y_label2 = pd.DataFrame( y_label2, index = unlabeled_df_train.index ).rename(columns = {0 : 'y_label2'})
labels_2 = y_labeled_score2.join(y_label2)[['y_labeled_score2', 'y_label2']]


rf = RandomForestClassifier(**parametrizacao['hyperparams'], random_state = 3)
sampled_df = X_train[novas_colunas_escolhidas].sample(frac = 0.97, random_state = 3)
y_sampled = y_train.loc[sampled_df.index]
rf.fit( sampled_df, y_sampled )
y_labeled_score3 = rf.predict_proba( unlabeled_df_train.join(lda_transform_erro_1_unlabeled)[novas_colunas_escolhidas] )
y_labeled_score3 = pd.DataFrame( y_labeled_score3, index = unlabeled_df_train.index ).rename(columns = {1 : 'y_labeled_score3'})
y_label3 = rf.predict( unlabeled_df_train.join(lda_transform_erro_1_unlabeled)[novas_colunas_escolhidas] )
y_label3 = pd.DataFrame( y_label3, index = unlabeled_df_train.index ).rename(columns = {0 : 'y_label3'})
labels_3 = y_labeled_score3.join(y_label3)[['y_labeled_score3', 'y_label3']]

labels_df = labels_1.join(labels_2).join(labels_3)
labels_df['target'] = -1
labels_df.loc[ labels_df.query('y_labeled_score1 > 0.8 and y_labeled_score2 > 0.8 and y_labeled_score3 > 0.8').index, ['target'] ] = 1
labels_df.loc[ labels_df.query('y_labeled_score1 <= 0.1 and y_labeled_score2 <= 0.1 and y_labeled_score3 <= 0.1').index, ['target'] ] = 0

unlabeled_df_train['target'] = labels_df['target']

# Abaixo é o resultado final - AUC se mantendo estavelmente ao redor de 0.922 ao ser testado com várias seeds

In [ ]:
rf = RandomForestClassifier(**parametrizacao['hyperparams'], random_state = 42)
rf.fit(X_train.append(unlabeled_df_train.join(lda_transform_erro_1_unlabeled).query('target != -1'))[novas_colunas_escolhidas], y_train.append(unlabeled_df_train.query('target != -1')['target']))


# Criacao das variaveis indicadoras de erro tipo 1 e 2 no teste

lda_transform_erro_1_test = lda_erro_1.transform(pls_erro_1.transform( X_test ) )
lda_transform_erro_1_test = pd.DataFrame( lda_transform_erro_1_test, index = X_test.index)[0].rename('lda_transform_erro_1_test')

###################################

y_score = rf.predict_proba(X_test.join(lda_transform_erro_1_test)[novas_colunas_escolhidas])
y_score = pd.DataFrame( y_score )[1]

roc_auc_score(y_test, y_score)

# Hipoteses futuras
* A utilização de um processo semi-supervisionado mais refinado pode trazer mais ganho do que o obtido, junto desses procedimentos
* A utilização de um intervalo no entorno da média de trust score das classificações erradas deve auxiliar como filtro no processo semi-supervisionado, diminuindo ruido adicionado retreino do dataset
* Em modelos com performance não tão alta, impacto de correção das piores zonas de erro do modelo deve ser consideravelmente maior, assim como a utilização de ensembles para isso (SVMs ou outros) que foquem nessa detecção a partir diversos pressupostos diferentes, em vez da geração de uma nova feature para ser adicionada no modelo antigo - como feito aqui, em vistas da discussão sobre interpretabilidade e busca de utilização desse método sem adicionar uma nova camada de complexidade sobre o problema
* Em casos mais complexos, como multiclasses, as zonas de confusão e erro sistematico do modelo devem ser mais espalhadas e com ganho de performance mais sensivel ao se utilizar esse procedimento
* Um estudo voltado a estabilidade temporal das variaveis pode ser importante para aplicação desse método com mais eficiencia, dado que a detecção de erro pode ser construida em cima de features que não irão manter sua capacidade de previsão pelo tempo - assim como variaveis utilizadas no próprio modelo

# Visualização Final

* Relembrando que essas visualizações podem ser realizadas a cada time step de forma a verificar como se encontra o comportamento do modelo - ou especificamente quando ocorrem variações consideraveis em sua performance no tempo, orientando os problemas recentes e como corrigir as alterações, seja de forma manual ou automática

In [ ]:
predicted_score_true_label = []

predicted_score_predicted_label = []
for i in range(0 , len(y_score)):
    predicted_score_true_label.append( y_score.tolist()[i] if y_score.tolist()[i] == 1 else 1 - y_score.tolist()[i] )
    predicted_score_predicted_label.append( max([y_score.tolist()[i], 1 - y_score.tolist()[i]]) )


from sklearn.decomposition import PCA

pca = PCA(n_components = 1)
pca.fit(X_train.drop(['erro_tipo_1', 'erro_tipo_2', 'lda_transform_erro_1_test', 'lda_transform_erro_2_test'], axis =1))
lens_pca = pca.transform(X_test)

lens = np.c_[lens_pca, y_test, predicted_score_true_label, predicted_score_predicted_label]

In [ ]:
graph = mapper.map(lens,
                   norm.transform(X_test),
                   cover=km.Cover(n_cubes= 5, perc_overlap=0.5),
                   remove_duplicate_nodes = True,
                   clusterer=sklearn.cluster.KMeans(n_clusters=2,
                                                    random_state= 42))


# Visualization
grafico = mapper.visualize(graph,
                 path_html="/kaggle/working/elliptic-dataset  Erro Analysis Prediction Score RESULTADO FINAL.html",
                 title=" Illicit Transaction - Elliptic Dataset - Heatmap Prediction Score - resultado final",
                 color_function= np.array(y_test))


# Visualization
grafico = mapper.visualize(graph,
                 path_html="/kaggle/working/elliptic-datasetn Error Analysis True Label RESULTADO FINAL.html",
                 title=" Illicit Transaction - Elliptic Dataset - Heatmap True Label - resultado final",
                 color_function= np.array(y_score))

In [ ]:
import nbconvert